In [1]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

def calculate_dcf(cashflows, discount_rate):
    """
    Calculate the DCF value based on cashflows and a discount rate.
    """
    years = np.arange(len(cashflows))
    discounted_cashflows = cashflows / (1 + discount_rate) ** years
    return discounted_cashflows.sum()

def plot_dcf(years=5, initial_cashflow=1000, growth_rate=0.05, discount_rate=0.1):
    """
    Visualize DCF based on the number of years, initial cashflow, growth rate, and discount rate.
    """
    cashflows = [initial_cashflow * ((1 + growth_rate) ** year) for year in range(years)]
    dcf_value = calculate_dcf(cashflows, discount_rate)

    # Plotting
    plt.figure(figsize=(10,6))
    plt.bar(range(years), cashflows, label="Projected Cash Flows")
    plt.axhline(y=dcf_value/years, color='r', linestyle='--', label="Average Discounted Cash Flows")
    plt.title(f"DCF Analysis (Value: ${dcf_value:.2f})")
    plt.xlabel("Year")
    plt.ylabel("Cash Flow")
    plt.legend()
    plt.grid(axis='y')
    plt.show()

# Interactive Widgets
years_slider = widgets.IntSlider(value=5, min=1, max=20, step=1, description='Years:')
initial_cashflow_slider = widgets.FloatSlider(value=1000, min=100, max=10000, step=50, description='Initial CF:')
growth_rate_slider = widgets.FloatSlider(value=0.05, min=-0.2, max=0.2, step=0.01, description='Growth Rate:')
discount_rate_slider = widgets.FloatSlider(value=0.1, min=0, max=0.2, step=0.01, description='Discount Rate:')

interactive_plot = widgets.interactive(
    plot_dcf,
    years=years_slider,
    initial_cashflow=initial_cashflow_slider,
    growth_rate=growth_rate_slider,
    discount_rate=discount_rate_slider
)

display(interactive_plot)


interactive(children=(IntSlider(value=5, description='Years:', max=20, min=1), FloatSlider(value=1000.0, descr…

In [2]:
import dash
from dash import html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import numpy as np

# Create the Dash app
app = dash.Dash(__name__)

# Layout of the Dash app
app.layout = html.Div([
    dcc.Graph(id='dcf-plot'),
    html.Div([
        dcc.Slider(
            id='years-slider',
            min=1,
            max=20,
            value=5,
            marks={i: str(i) for i in range(1, 21)},
            step=1
        ),
        html.Label('Years'),
        
        dcc.Slider(
            id='initial-cf-slider',
            min=100,
            max=10000,
            value=1000,
            marks={i: str(i) for i in range(100, 10001, 1000)},
            step=50
        ),
        html.Label('Initial CF'),
        
        dcc.Slider(
            id='growth-rate-slider',
            min=-0.2,
            max=0.2,
            value=0.05,
            marks={i: f'{i:.2f}' for i in np.linspace(-0.2, 0.2, 9)},
            step=0.01
        ),
        html.Label('Growth Rate'),
        
        dcc.Slider(
            id='discount-rate-slider',
            min=0,
            max=0.2,
            value=0.1,
            marks={i: f'{i:.2f}' for i in np.linspace(0, 0.2, 9)},
            step=0.01
        ),
        html.Label('Discount Rate')
    ])
])

@app.callback(
    Output('dcf-plot', 'figure'),
    [Input('years-slider', 'value'),
     Input('initial-cf-slider', 'value'),
     Input('growth-rate-slider', 'value'),
     Input('discount-rate-slider', 'value')]
)
def update_plot(years, initial_cashflow, growth_rate, discount_rate):
    cashflows = [initial_cashflow * ((1 + growth_rate) ** year) for year in range(years)]
    dcf_value = sum(cashflows) / (1 + discount_rate) ** np.arange(years)

    trace1 = go.Bar(x=list(range(years)), y=cashflows, name='Projected Cash Flows')
    trace2 = go.Scatter(x=list(range(years)), y=[dcf_value.mean()] * years, mode='lines', name='Average Discounted Cash Flows', line=dict(dash='dash', color='red'))
    
    layout = go.Layout(
        title=f"DCF Analysis (Value: ${dcf_value.sum():.2f})",
        xaxis=dict(title="Year"),
        yaxis=dict(title="Cash Flow"),
        hovermode="x unified"
    )
    
    return {"data": [trace1, trace2], "layout": layout}

if __name__ == '__main__':
    app.run_server(debug=True)


/var/folders/65/57wn7ttx42713rnb3qc3k2ph0000gn/T/ipykernel_89082/1409045097.py:3: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
